Cell 1: Install and import

In [ ]:
!pip install einops pandas scikit-learn matplotlib
!pip install torch

import torch
import torch.nn as nn
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset


Cell 2: Load & process data

In [ ]:
df = pd.read_csv('/content/merged_sentiment_price.csv')
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date')

# Create multivariate input
df['price_change'] = df['close'].pct_change()
df['sentiment_smooth'] = df['sentiment'].rolling(3).mean()
df = df.dropna().reset_index(drop=True)

features = ['close', 'price_change', 'sentiment_smooth']
scaler = StandardScaler()
scaled = scaler.fit_transform(df[features])

SEQ_LEN = 30
PRED_LEN = 7

def create_timeseries(data, seq_len, pred_len):
    X, y = [], []
    for i in range(len(data) - seq_len - pred_len):
        X.append(data[i:i+seq_len])
        y.append(data[i+seq_len:i+seq_len+pred_len, 0])  # forecast 'close'
    return np.array(X), np.array(y)

X, y = create_timeseries(scaled, SEQ_LEN, PRED_LEN)


Cell 3: Split and load to DataLoader

In [ ]:
train_size = int(0.8 * len(X))
X_train, y_train = X[:train_size], y[:train_size]
X_test, y_test = X[train_size:], y[train_size:]

train_ds = TensorDataset(torch.tensor(X_train, dtype=torch.float32),
                         torch.tensor(y_train, dtype=torch.float32))
test_ds = TensorDataset(torch.tensor(X_test, dtype=torch.float32),
                        torch.tensor(y_test, dtype=torch.float32))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
test_loader = DataLoader(test_ds, batch_size=32)


Cell 4: Define TimesNet-like model

In [ ]:
class TimesNet(nn.Module):
    def __init__(self, input_size, seq_len, pred_len):
        super().__init__()
        self.encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=input_size, nhead=2),
            num_layers=2
        )
        self.flatten = nn.Flatten()
        self.linear = nn.Linear(seq_len * input_size, pred_len)

    def forward(self, x):
        x = self.encoder(x)
        x = self.flatten(x)
        return self.linear(x)

model = TimesNet(input_size=X.shape[2], seq_len=SEQ_LEN, pred_len=PRED_LEN)
loss_fn = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


Cell 5: Train the model

In [ ]:
EPOCHS = 10
for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    for xb, yb in train_loader:
        preds = model(xb)
        loss = loss_fn(preds, yb)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")


Cell 6: Forecast & plot

In [ ]:
model.eval()
with torch.no_grad():
    sample_input = torch.tensor(X_test[0:1], dtype=torch.float32)
    prediction = model(sample_input).numpy()[0]

true = y_test[0]
plt.plot(range(PRED_LEN), true, label="Actual")
plt.plot(range(PRED_LEN), prediction, label="Predicted")
plt.legend()
plt.title("7-day Forecast")
plt.show()


Cell 7: Save the model

In [ ]:
torch.save(model.state_dict(), "timesnet_model.pt")

Cell 8: Optional download

In [ ]:
from google.colab import files
files.download("timesnet_model.pt")
